In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium import LayerControl

def create_other_features_gdf(fp):
    df = pd.read_csv(fp)
    geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    return gdf

### Read in footfall sensors

In [2]:
sensors_gdf = pd.read_csv("../Cleaned_data/SensorData/sensor_locations.csv")
sensors_gdf.rename(columns={'sensor_description': 'Name'}, inplace = True)

### Read in features

In [3]:
buildings = create_other_features_gdf("../Cleaned_data/SpatialFeatures/buildings_clean.csv")
landmarks = create_other_features_gdf("../Cleaned_data/SpatialFeatures/landmarks_clean.csv")
furniture = create_other_features_gdf("../Cleaned_data/SpatialFeatures/street_inf_clean.csv")
memorials = create_other_features_gdf("../Cleaned_data/SpatialFeatures/memorials_clean.csv")
fountains = create_other_features_gdf("../Cleaned_data/SpatialFeatures/fountains_clean.csv")
all_transport_stops = create_other_features_gdf("../Cleaned_data/SpatialFeatures/transport_stops_clean.csv")

### Plot features

In [4]:
# Plot the basic map        
f = folium.Figure(width=800, height=600)
melbourne_map = folium.Map(location=[-37.81426902667574, 144.97034862684487], zoom_start=14, control_scale=True, min_zoom = 10).add_to(f)
             
# Add sensors
sensors_fg=folium.FeatureGroup(name='Sensors', show=False)
melbourne_map.add_child(sensors_fg)
for index, location_info in sensors_gdf.iterrows():
    folium.Marker([location_info["Latitude"], location_info["Longitude"]], radius=5,
                  popup = location_info['Name']).add_to(sensors_fg)   

# Add bus stops
bus_stops_fg=folium.FeatureGroup(name='Bus Stops', show=False)
melbourne_map.add_child(bus_stops_fg)
for index, location_info in all_transport_stops[all_transport_stops['Type']=='bus_stop'].iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=5, fill = True,
                        color='green', fill_color="green", popup = 'bus stop',fill_opacity=1).add_to(bus_stops_fg)    


# Add tram stops
tram_stops_fg=folium.FeatureGroup(name='Tram Stops', show=False)
melbourne_map.add_child(tram_stops_fg)
for index, location_info in all_transport_stops[all_transport_stops['Type']=='tram_stop'].iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=5, fill = True,
                        color='purple', fill_color="purple", popup = 'tram stop',fill_opacity=1).add_to(tram_stops_fg)      
    
    
# Add metro stations
metro_stations_fg =folium.FeatureGroup(name='Metro Stops', show=False)
melbourne_map.add_child(metro_stations_fg)
for index, location_info in all_transport_stops[all_transport_stops['Type']=='metro_stop'].iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=5, fill = True,
                        color='blue', fill_color="blue", popup = 'Metro station',fill_opacity=1).add_to(metro_stations_fg)  
    
    
# Add layer control
melbourne_map.add_child(LayerControl("topright", collapsed = False))

melbourne_map

In [5]:
# Plot the basic map        
f = folium.Figure(width=800, height=600)
melbourne_map = folium.Map(location=[-37.81426902667574, 144.97034862684487], zoom_start=14, control_scale=True, min_zoom = 10).add_to(f)
             
# Add bus stops
bus_stops_fg=folium.FeatureGroup(name='Fountains', show=False)
melbourne_map.add_child(bus_stops_fg)
for index, location_info in fountains.iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=5, fill = True,
                        color='green', fill_color="green", popup = 'Fountains',fill_opacity=1).add_to(bus_stops_fg)    

# Add metro stations
fountain_fg =folium.FeatureGroup(name='Fountain', show=False)
melbourne_map.add_child(fountain_fg)
for index, location_info in furniture[furniture['feature']=='Drinking Fountain'].iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=5, fill = True,
                        color='blue', fill_color="blue", popup = 'Fountain (furniture)',fill_opacity=1).add_to(fountain_fg)  
    
# Add layer control
melbourne_map.add_child(LayerControl("topright", collapsed = False))
melbourne_map